## Part 1: Preprocessing

In [3]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [4]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [5]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()


,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [23]:
# Create a list of at least 10 column names to use as X data
X = ['Education', 'Age', 'DistanceFromHome','JobSatisfaction','OverTime', 'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany','YearsSinceLastPromotion','NumCompaniesWorked']


# Create X_df using your selected columns
X_df = attrition_df[X]
X_df.head()

# Show the data types for X_df
X_df.dtypes
X_df['OverTime'] = X_df['OverTime'].map({'Yes': 1, 'No': 0}).astype("int64")


<ipython-input-23-44f2f937f682>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_df['OverTime'] = X_df['OverTime'].map({'Yes': 1, 'No': 0}).astype("int64")


In [24]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=78)

In [25]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X = pd.get_dummies(X_df)
X.head()

,Education,Age,DistanceFromHome,JobSatisfaction,OverTime,StockOptionLevel,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion,NumCompaniesWorked
0,2,41,1,4,1,0,1,6,0,8
1,1,49,8,2,0,1,3,10,1,1
2,2,37,2,3,1,0,3,0,0,6
3,4,33,3,3,1,0,3,8,3,1
4,1,27,2,2,0,1,3,2,2,9


In [26]:
# Create a StandardScaler
X_scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [33]:
# Create a OneHotEncoder for the Department column
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit the encoder to the training data
encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
encoder_train = encoder.transform(y_train[['Department']])
encoder_test = encoder.transform(y_test[['Department']])



In [34]:
# Create a OneHotEncoder for the Attrition column
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit the encoder to the training data
encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
encoder_train = encoder.transform(y_train[['Attrition']])
encoder_test = encoder.transform(y_test[['Attrition']])


## Create, Compile, and Train the Model

In [37]:
# Find the number of columns in the X training data
X_train.shape[1]

# Create the input layer
input_nodes = len(X_train.columns)


# Create at least two shared layers
shared_layer_1 = layers.Dense(units=64, activation='relu')
shared_layer_2 = layers.Dense(units=32, activation='relu')


(1102, 10)

In [ ]:
# Create a branch for Department
# with a hidden layer and an output layer
shared_input = layers.Input(shape=(input_nodes,))
shared_x = shared_layer_1(shared_input)
shared_x = shared_layer_2(shared_x)
# Create the hidden layer
department_hidden = layers.Dense(units=64, activation='relu')(shared_x)

# Create the output layer
department_output = layers.Dense(units=2, activation='softmax')(department_hidden)



In [ ]:
# Create a branch for Attrition
# with a hidden layer and an output layer
s

# Create the hidden layer


# Create the output layer



In [ ]:
# Create the model


# Compile the model


# Summarize the model


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input (InputLayer)          [(None, 10)]                 0         []                            
                                                                                                  
 shared1 (Dense)             (None, 64)                   704       ['input[0][0]']               
                                                                                                  
 shared2 (Dense)             (None, 128)                  8320      ['shared1[0][0]']             
                                                                                                  
 department_hidden (Dense)   (None, 32)                   4128      ['shared2[0][0]']             
                                                                                              

In [ ]:
# Train the model



Epoch 1/100
35/35 [==============================] - 14s 25ms/step - loss: 1.4162 - department_output_loss: 0.8374 - attrition_output_loss: 0.5787 - department_output_accuracy: 0.6316 - attrition_output_accuracy: 0.7024
Epoch 2/100
35/35 [==============================] - 1s 27ms/step - loss: 1.2070 - department_output_loss: 0.7661 - attrition_output_loss: 0.4408 - department_output_accuracy: 0.6570 - attrition_output_accuracy: 0.8403
Epoch 3/100
35/35 [==============================] - 0s 13ms/step - loss: 1.1520 - department_output_loss: 0.7539 - attrition_output_loss: 0.3981 - department_output_accuracy: 0.6570 - attrition_output_accuracy: 0.8403
Epoch 4/100
35/35 [==============================] - 0s 14ms/step - loss: 1.1231 - department_output_loss: 0.7452 - attrition_output_loss: 0.3779 - department_output_accuracy: 0.6588 - attrition_output_accuracy: 0.8475
Epoch 5/100
35/35 [==============================] - 0s 12ms/step - loss: 1.1031 - department_output_loss: 0.7342 - attriti

In [ ]:
# Evaluate the model with the testing data


12/12 [==============================] - 0s 3ms/step - loss: 4.1624 - department_output_loss: 3.1143 - attrition_output_loss: 1.0481 - department_output_accuracy: 0.5272 - attrition_output_accuracy: 0.8261


[4.162380218505859,
 3.114327907562256,
 1.0480519533157349,
 0.5271739363670349,
 0.8260869383811951]

In [ ]:
# Print the accuracy for both department and attrition


Department predictions accuracy: 0.5271739363670349
Attrition predictions accuracy: 0.8260869383811951


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1.
2.
3.